# Actions to create buildings (aka Construction)
Populations take actions, one kind of action is the building. 


In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio



# mapping to the modules that make the app
sys.path.insert(0, "../..")


In [2]:
from app.creators import universe
from app.functions import maths
from app.functions import configurations
from app.objects import time as t

from notebooks.helpers import test_queries

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import app.models as models

c = models.CosmosdbClient()

executing local windows deployment


**Note** that there is a doc within the yaml in `explanation`

In [4]:
conf = configurations.get_building_configurations()
conf['building']['explanation']

[{'type': "all buildings have a 'type'/'label' of building"},
 {'name': "name_with_underscore. UI can remove the '_' at display time."},
 {'augments_resource': 'OPTIONAL augments a specific resource, creates it if it does not exist'},
 {'built_by': 'type of object that can build the building'},
 {'owned_by': 'type of object that can own/maintain this building, assuming a relationhsip to that object'},
 {'changes': 'stats changed by the owner'},
 {'faction_augments': 'if owned by a pop in the faction,  raises the stat for all pops in factions'},
 {'effort': 'time in `universal time units` that the action will take to complete'},
 {'requires_attr': 'required attributes of the agent. Determines if the agent can do the action.'},
 {'render_type': "guides the rendering in buildings.js. Available types: 'block'"}]

In [13]:
conf['building']['buildings'][0]

{'name': 'farmland',
 'description': 'Generates organic foodstuffs, populations will consume food before consuming natural resources',
 'planet_requirements': {'isHabitable': 'true'},
 'augments_resource': {'vegitable_foodstuff': 10},
 'changes': {'wealth': -0.01},
 'owned_by': 'pop',
 'effort': 1,
 'render_type': 'block'}

`Time` fetches all of the relevant actions

In [5]:
time = t.Time(c)
time.get_current_UTU()
time.get_global_actions()

## Adding a building action to a pop, for testing. 

Go to the app and create a building action. Then run this notebook. You should be able to see that a `construction` action has been created for that pop. 

This is done in `templates\app\ajax`, but for testing it's best to do it in the UI. 

In [6]:
actions_df = pd.DataFrame(time.actions)
actions_df

,action,job,agent
0,"{'type': 'patriot_propoganda', 'applies_to': '...","{'status': 'pending', 'username': 'BillmanLoca...","{'isIdle': 'false', 'name': 'Waquimac Mos', 'o..."
1,"{'type': 'construction', 'comment': 'construct...","{'status': 'pending', 'username': 'BillmanLoca...","{'isIdle': 'false', 'name': 'Danne Gorskbiley'..."
2,"{'type': 'construction', 'comment': 'construct...","{'status': 'pending', 'username': 'BillmanLoca...","{'isIdle': 'false', 'name': 'Waquimac Lamdon',..."
3,"{'type': 'individual_education', 'requires_att...","{'status': 'pending', 'username': 'BillmanLoca...","{'isIdle': 'false', 'name': 'Waquimac Tsugidra..."
4,"{'type': 'healthcare_initiatives', 'applies_to...","{'status': 'pending', 'username': 'BillmanLoca...","{'isIdle': 'false', 'name': 'Danne Perpennasmo..."


from that dataframe we are going to get a list of action objects 

In [7]:
validActionCounter = 0 
actions = [t.Action(c,actions_df.loc[i]) for i  in actions_df.index]
actions

[< (Waquimac Mos: 1258976461150) -takingAction:1013-> (patriot propoganda) >,
 < (Danne Gorskbiley: 2194544643353) -takingAction:1013-> (construction) >,
 < (Waquimac Lamdon: 7400632371289) -takingAction:1013-> (construction) >,
 < (Waquimac Tsugidra: 0885306882212) -takingAction:1014-> (individual education) >,
 < (Danne Perpennasmosgikrahas: 4617413097199) -takingAction:1014-> (healthcare initiatives) >]

In [16]:
a.action

{'type': 'construction',
 'comment': 'constructing a farmland',
 'effort': 1,
 'applies_to': 'pop',
 'owned_by': 'pop',
 'building': 'farmland',
 'created_at': 1012,
 'objid': '3167210905929',
 'username': 'BillmanLocal2',
 'objtype': 'action'}

In [17]:
for a in actions:
    a.add_updates_to_c(time)

< (Danne Gorskbiley: 2194544643353) -takingAction:1013-> (construction) > is a building action


AttributeError: 'Action' object has no attribute 'make_building'

In [12]:
actions[1].c.stack

["g.V().has('objid','1258976461150').property('factionLoyalty',0.599).property('conformity',0.52).property('aggression',0.426)",
 "g.V().has('objid','1258976461150').outE('takingAction').has('actionType','patriot_propoganda').has('weight','1013').property('status','resolved')",
 "g.V().has('objid','1258976461150').property('isIdle','true')",
 "g.V().has('objid','2194544643353').outE('takingAction').has('actionType','construction').has('weight','1013').property('status','resolved')",
 "g.V().has('objid','2194544643353').property('isIdle','true')",
 "g.V().has('objid','7400632371289').outE('takingAction').has('actionType','construction').has('weight','1013').property('status','resolved')",
 "g.V().has('objid','7400632371289').property('isIdle','true')",
 "g.V().has('objid','0885306882212').property('wealth',0.5308).property('literacy',0.473).property('factionLoyalty',0.437)",
 "g.V().has('objid','0885306882212').outE('takingAction').has('actionType','individual_education').has('weight','

In [13]:
actions[3].c.stack

["g.V().has('objid','1258976461150').property('factionLoyalty',0.599).property('conformity',0.52).property('aggression',0.426)",
 "g.V().has('objid','1258976461150').outE('takingAction').has('actionType','patriot_propoganda').has('weight','1013').property('status','resolved')",
 "g.V().has('objid','1258976461150').property('isIdle','true')",
 "g.V().has('objid','2194544643353').outE('takingAction').has('actionType','construction').has('weight','1013').property('status','resolved')",
 "g.V().has('objid','2194544643353').property('isIdle','true')",
 "g.V().has('objid','7400632371289').outE('takingAction').has('actionType','construction').has('weight','1013').property('status','resolved')",
 "g.V().has('objid','7400632371289').property('isIdle','true')",
 "g.V().has('objid','0885306882212').property('wealth',0.5308).property('literacy',0.473).property('factionLoyalty',0.437)",
 "g.V().has('objid','0885306882212').outE('takingAction').has('actionType','individual_education').has('weight','

In [10]:
building_query = f"""
g.V().has('objid','{pop['objid']}').as('pop')
    .in('owns').as('building')
    .path()
        .by(valueMap('objid','name'))
        .by(valueMap('objid','name','changes','augments_resource','planet_requirements','description','render_type'))
"""

c.run_query(building_query)
c.res

[{'labels': [['pop'], ['building']],
  'objects': [{'objid': ['7467985401518'], 'name': ['Obattla Magor']},
   {'objid': ['8925307376683'],
    'name': ['farmland'],
    'changes': ['{wealth: -0.01}'],
    'augments_resource': ['{vegitable_foodstuff: 10}'],
    'planet_requirements': ['{isHabitable: true}'],
    'description': ['Generates organic foodstuffs, populations will consume food before consuming natural resources'],
    'render_type': ['block']}]}]

In [ ]:
##